In [1]:
from math import log
from rocketcea.cea_obj_w_units import CEA_Obj

In [2]:
C = CEA_Obj(oxName="LOX", fuelName="RP-1")

Wpayload = 8500
Wstg = 5106
Wpropellant = 45920

Winit = Wstg + Wpropellant + Wpayload
Wfinal = Winit - Wpropellant

def show_deltaV( Pc=475.0, eps=84.0, MR=5.88 ):
    IspVac = C.get_Isp(Pc=Pc, MR=MR, eps=eps)
    IspDel = 0.969 * IspVac
    deltaV = 32.174 * IspDel * log( Winit / Wfinal ) # ft/sec
    print( '%8.1f %8.1f    %8.2f    %8.1f     %8.1f        %8.1f     '%(Pc, eps, MR, IspVac, IspDel, deltaV))

print(' Pc(psia) AreaRatio  MixtureRatio   IspVac(sec)  IspDel(sec)   deltaV(ft/sec)')
show_deltaV(Pc=475, eps=120)


 Pc(psia) AreaRatio  MixtureRatio   IspVac(sec)  IspDel(sec)   deltaV(ft/sec)
   475.0    120.0        5.88       322.1        312.2         14822.7     


# Missão

In [3]:
deltav_mission = 12000
payload_weight = 5000
payload_height = 6.7
payload_diameter = 4.6

# Parâmetros Fixos

In [4]:
n_stages = 2
deltav_stages = {1: 3500, 2: 8500}
stage_radii = {1: 1.8, 2: 1.8}
oxidizer = "LOX"
fuel = "RP-1"
engine_cycle = "Gas Generator"
chamber_pressure = {1: 97, 2: 97}
coef_estrutual = {1: 0.06, 2: 0.05}
max_n_engines = {1: 15, 2: 1}


# Fronteira dos parâmetros

In [5]:
mixture_ratior_border = {"begin": 1.5, "end": 3.5}
throat_diameter_border = {"begin": 0.2, "end": 0.3}

# Restrições

In [6]:
min_acceleration = {1: 1.3, 2: 0.8}
max_ld_ration = {1: 25, 2: 25}
molar_masses = {"LOX":32, "RP-1":175}

# -------

In [106]:
import math

class Engine:
    def __init__(self,
                 fuel_name:str,
                 ox_name:str,
                 mr:float,
                 pc:float,
                 nozzle_diam:float,
                 eps:float):
        """
        fuel_name: 
        ox_name (oxidizer name):
        mr (mixture ratio):
        nozzle_diam (nozzle throat diameter):
        esp (nozzle expansion ratio)
        """
        
        self.fuel_name = fuel_name
        self.ox_name = ox_name
        self.mr = mr
        self.pc = pc
        self.nozzle_diam = nozzle_diam
        self.eps = eps

        self.R_universal = 8.314462 # Constante universal dos gases

        self.C = CEA_Obj(oxName=self.ox_name, 
                         fuelName=self.fuel_name,
                         isp_units='sec',
                         cstar_units='m/s', 
                         pressure_units='MPa', 
                         temperature_units='K', 
                         sonic_velocity_units='m/s', 
                         enthalpy_units='J/g', 
                         density_units='kg/m^3', 
                         specific_heat_units='J/kg-K', 
                         viscosity_units='millipoise', 
                         thermal_cond_units='mcal/cm-K-s')


    def get_F_vac(self):
        #IspVac, Pe_vac = self.C.estimate_Ambient_Isp(Pc=self.pc, MR=self.mr, eps=self.eps, Pamb=0.00001)
        IspVac = self.C.get_Isp(Pc=self.pc, MR=self.mr, eps=self.eps)
        IspAmb, Pe_amb = self.C.estimate_Ambient_Isp(Pc=self.pc,Pamb=1e5, MR=self.mr, eps=self.eps)
        temperature_chamber, temperature_throat, temperature_exit = self.C.get_Temperatures(Pc=self.pc, MR=self.mr, eps=self.eps)
        pc_pe = self.C.get_PcOvPe(Pc=self.pc, MR=self.mr, eps=self.eps)
        chamber_velocity, throat_velocity, exit_velocity = self.C.get_SonicVelocities(Pc=self.pc, MR=self.mr, eps=self.eps)
        chamber_density, throat_density, exit_density = self.C.get_Densities(Pc=self.pc, MR=self.mr, eps=self.eps)
        exit_mach = self.C.get_MachNumber(Pc=self.pc, MR=self.mr, eps=self.eps)
        cf_vac, cf_sea, cf_mode = self.C.get_PambCf(Pc=self.pc, MR=self.mr, eps=self.eps, Pamb=1)#thrust coefficient

        nozzle_throat_area = math.pi*((self.nozzle_diam/2) **2)
        nozzle_exit_area = nozzle_throat_area * self.eps
        pe = 1/pc_pe * self.pc

        exit_speed = exit_mach * exit_velocity/1000
        mass_flow = exit_density/1000 * nozzle_exit_area * exit_speed
        thrust_vacuum = mass_flow * exit_velocity + pe  * nozzle_exit_area
        thrust_sea = mass_flow * exit_velocity + (pe-1e5)* nozzle_exit_area

        thrust_vacuum = cf_vac * self.pc * nozzle_throat_area
        thrust_sea = cf_sea * self.pc * nozzle_throat_area


        print(nozzle_exit_area)
        print("Isp Vac: " + str(IspVac),"Isp Sea: " + str(IspAmb)) 
        print("Mass flow: " + str(mass_flow))
        print(f"Thrust Vac {thrust_vacuum}", f"Thrust Sea {thrust_sea}")

        return



engine = Engine(fuel_name="RP-1", 
                ox_name="LOX",
                mr=2.3,
                pc=9.7*1e6 ,
                nozzle_diam=0.23125,
                eps=165)

engine.get_F_vac()



6.930065005430962
Isp Vac: 350.60767346744154 Isp Sea: 136.35328867508358
Mass flow: 298.4472616654054
Thrust Vac 973818.3344951972 Thrust Sea 1026147.6426383896


In [ ]:
engine = Engine(fuel_name="RP-1", 
                ox_name="LOX",
                mr=2.3,
                pc=9.7*1e6 ,
                nozzle_diam=0.23125,
                eps=165)

engine.get_F_vac()


# Referências bibliográficas

- https://cearun.grc.nasa.gov/intro.html
- https://www.grc.nasa.gov/www/k-12/airplane/mflchk.html
- https://rocketcea.readthedocs.io/en/latest/functions.html
- http://www.braeunig.us/space/index.htm
- https://www.grc.nasa.gov/www/k-12/airplane/rktthsum.html
- https://www.grc.nasa.gov/www/k-12/rocket/nozzle.html
- http://mae-nas.eng.usu.edu/MAE_5420_Web/section5/section.5.3.pdf
- http://mae-nas.eng.usu.edu/MAE_5540_Web/propulsion_systems/MAE_5540_2022.html


# Motor Merlin-1D - Especificações
- https://www.wevolver.com/specs/merlin-engine-merlin-1d-falcon-9-falcon-heavy
- http://www.b14643.de/Spacerockets_2/United_States_1/Falcon-9/Merlin/index.htm

# Implementacoes
- https://proptools.readthedocs.io/en/latest/
- https://github.com/bluedack-space/OnlineRocketDesigner/blob/d8bb006de7a5c596b64008793a9bc778327d9143/rocketEngineHandler.py#L65

